### Use Python LightGBM and Optuna hyper parameter tuning to build model and deploy with KNIME Python nodes


<img src="../KNIME_loves_lightgbm_optuna.png" width="600">


##### KNIME workflows for Binary Classifications
https://hub.knime.com/-/spaces/-/~GABT_OgeoWxWJW9P/current-state/

https://hub.knime.com/-/spaces/-/~0dXvsD0vMrv_w6Fw/current-state/


##### MEDIUM Blog: Hyperparameter optimization for LightGBM — wrapped in KNIME nodes

https://medium.com/p/ddb7ae1d7e2

###### Optuna

https://optuna.readthedocs.io/en/stable/index.html

###### additional links and texts

https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html#for-better-accuracy

https://stackoverflow.com/questions/67080149/xgboost-error-when-categorical-type-is-supplied-dmatrix-parameter-enable-cat


https://towardsdatascience.com/how-to-beat-the-heck-out-of-xgboost-with-lightgbm-comprehensive-tutorial-5eba52195997

https://towardsdatascience.com/kagglers-guide-to-lightgbm-hyperparameter-tuning-with-optuna-in-2021-ed048d9838b5


https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f

some parameters have been discussed with ChatGPT ....

In [ ]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import json
import pickle
import logging

In [ ]:
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation, record_evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, auc, average_precision_score, precision_recall_curve

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
import matplotlib.pyplot as plt
import plotly
import matplotlib
import kaleido

In [ ]:
# http://strftime.org'
import time
var_timestamp_day = "{}".format(time.strftime("%Y%m%d"))
# flow_variables['var_timestamp_day'] = var_timestamp_day
print("var_timestamp_day: ", var_timestamp_day)

var_timestamp_time = "{}h".format(time.strftime("%H%M"))
# flow_variables['var_timestamp_time'] = var_timestamp_time
print("var_timestamp_time: ", var_timestamp_time)

# _edit: if you want to have another model name
var_model_name = "LightGBM_Optuna_Classification"
# flow_variables['var_model_name'] = var_model_name

var_model_name_full = var_model_name + "_" + var_timestamp_day  + "_" + var_timestamp_time + "_jupyter"
# flow_variables['var_model_name_full'] = var_model_name_full
print("var_model_name_full: ", var_model_name_full)

# if you dp not want to store the files in the working directory
var_path_data = "../result/"
var_path_model = "../model/"

In [ ]:
data      = pq.read_table("../train.parquet").to_pandas()
data_test = pq.read_table("../test.parquet").to_pandas()

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data_test = data_test.reset_index(drop=True)

In [ ]:
excluded_features = ['row_id']
label = ['Target']
# features = [feat for feat in data.columns if feat not in excluded_features and not feat==label]
features = [feat for feat in data.columns if feat not in excluded_features and feat not in label]

num_cols = data[features].select_dtypes(include='number').columns.tolist()
cat_cols = data[features].select_dtypes(exclude='number').columns.tolist()

rest_cols = [feat for feat in data.columns if feat not in cat_cols]

print(f'''{"data shape:":20} {data.shape}
{"data[features] shape:":20} {data[features].shape}
categorical columns: {cat_cols}
numerical columns: {num_cols}
feature columns: {features}
rest columns: {rest_cols}''')

# THX David Gutmann 

In [ ]:
print(data.dtypes)

In [ ]:
data[cat_cols] = data[cat_cols].astype('category')

In [ ]:
data[label] = data[label].astype('int32')

### Normalize by Log(), Decimal or Robust Scaling

In [ ]:
# Define the number of decimal places for the scaling
decimal_places_n = 2

In [ ]:
def decimal_scaling(input_df, cols_to_normalize, decimal_places):
    """
    Apply decimal scaling to the selected columns in the input dataframe and return a new dataframe with the scaled values.
    
    Parameters:
    input_df (pandas.DataFrame): The input dataframe to be scaled.
    cols_to_normalize (list): A list of column names to be scaled.
    decimal_places (int): The number of decimal places to scale the values.
    
    Returns:
    pandas.DataFrame: A new dataframe with the scaled values.
    """
    output_df = input_df.copy()
    for col in cols_to_normalize:
        max_value = input_df[col].max()
        min_value = input_df[col][input_df[col] > 0].min()
        scaling_factor = 10 ** (len(str(int(max_value))) - decimal_places)
        output_df[col] = input_df[col].apply(lambda x: (x - min_value) / scaling_factor)
    return output_df


In [ ]:
# Apply decimal scaling to the input dataframe and store the result in the output dataframe
data_normalized = decimal_scaling(input_df=data, cols_to_normalize=num_cols, decimal_places=decimal_places_n)

In [ ]:
data_normalized.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def log_scaling(input_df, cols_to_transform, cols_to_normalize, decimal_places=2):
    """
    Apply log transformation and Min-Max scaling to the selected columns in the input dataframe and return a new dataframe with the transformed and normalized values.
    
    Parameters:
    input_df (pandas.DataFrame): The input dataframe to be transformed and scaled.
    cols_to_transform (list): A list of column names to be transformed with log.
    cols_to_normalize (list): A list of column names to be normalized.
    decimal_places (int, optional): The number of decimal places to scale the values. Default is 2.
    
    Returns:
    pandas.DataFrame: A new dataframe with the transformed and normalized values.
    """
    output_df = input_df.copy()
    for col in cols_to_transform:
        if output_df[col].min() <= 0:
            min_value = output_df[col][output_df[col] > 0].min()
            output_df[col] = output_df[col].apply(lambda x: np.log(x - min_value + 1))
        else:
            output_df[col] = output_df[col].apply(lambda x: np.log(x))
    scaler = MinMaxScaler(feature_range=(0, 100))
    output_df[cols_to_normalize] = scaler.fit_transform(output_df[cols_to_normalize]).round(decimal_places)
    return output_df

In [ ]:
data_normalized = log_scaling(input_df=data, cols_to_transform=num_cols, cols_to_normalize=num_cols)

In [ ]:
data_normalized.head()

In [ ]:
from sklearn.preprocessing import RobustScaler

def robust_scaling(input_df, cols_to_scale, quantile_range=(25.0, 75.0)):
    """
    Apply robust scaling to the selected columns in the input dataframe and return a new dataframe with the scaled values.
    
    Parameters:
    input_df (pandas.DataFrame): The input dataframe to be scaled.
    cols_to_scale (list): A list of column names to be scaled.
    quantile_range (tuple, optional): The quantile range used to compute the scale. Default is (25.0, 75.0).
    
    Returns:
    pandas.DataFrame: A new dataframe with the scaled values.
    """
    output_df = input_df.copy()
    scaler = RobustScaler(with_centering=True, with_scaling=True, quantile_range=quantile_range)
    output_df[cols_to_scale] = scaler.fit_transform(input_df[cols_to_scale])
    return output_df

In [ ]:
# Apply robust scaling to the selected columns
data_normalized = robust_scaling(input_df=data, cols_to_scale=num_cols)

In [ ]:
data_normalized.head(50)

In [ ]:
print(df_normalized.dtypes)

In [ ]:
# split training data into X and y
X = data[features]
y = data[label]

In [ ]:
# split training data into X and y
X = data[features]
y = data[label]

In [ ]:
X.head()

In [ ]:
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [ ]:
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

In [ ]:
 # Convert data into LightGBM dataset format
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_val = lgb.Dataset(X_test, label=y_test, reference=lgb_train)

callbacks = [early_stopping(stopping_rounds=50, eval_metric="aucpr", first_metric_only=True),
             log_evaluation(log_filename=None, period=100, show_stdv=True)]

model = lgb.train(params, lgb_train, valid_sets=[lgb_val], callbacks=callbacks, num_boost_round=var_n_boost_round)


In [ ]:
# number of iterations
var_n_boost_round = 200
    
def objective(trial):
    params = {
        'objective': 'binary',  # The task is binary classification.
        'metric': 'aucpr',  # The evaluation metric is AUCPR.
        'n_estimators': trial.suggest_int('n_estimators', 500, 1000, step=100),  # The number of boosting rounds.
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),  # The type of boosting algorithm.
        # 'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),  # The class weight balance.
        # 'importance_type': trial.suggest_categorical('importance_type', ['split', 'gain']),  # The type of feature importance metric.
        'num_leaves': trial.suggest_int('num_leaves', 25, 500),  # The maximum number of leaves in one tree.
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),  # The learning rate of the boosting process.
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),  # The fraction of features to be randomly sampled for each tree.
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),  # The fraction of data to be randomly sampled for each tree.
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),  # The frequency of bagging.
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0),  # L1 regularization term on weights.
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0),  # L2 regularization term on weights.
        # 'reg_alpha': trial.suggest_float('reg_alpha', 0, 2),
        # 'reg_lambda': trial.suggest_float('reg_lambda', 0, 2),
        # You need to set `feature_pre_filter=false` to dynamically change the `min_data_in_leaf`.
        # 'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 500), # The minimum number of samples required to be at a leaf node.
        'max_depth': trial.suggest_int('max_depth', 6, 16),  # The maximum depth of a tree.
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0, 1.0),  # The minimum gain to split a leaf node.
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),  # The fraction of data to be used for each tree.
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),  # The fraction of features to be used for each tree.
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0),  # L1 regularization term on leaf weights.
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0),  # L2 regularization term on leaf weights.
        # 'is_unbalance': trial.suggest_categorical('is_unbalance', [True, False]),  # Whether to balance the positive and negative weights.
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),  # The ratio of the sum of the weights of the negative class samples to the positive class samples.
        'seed': 42,  # The random seed.
        'n_jobs': -1  # Number of CPU threads to use for parallel execution, -1 means use all available CPU threads
    }
 
    # You need to set `feature_pre_filter=false` to dynamically change the `min_data_in_leaf`.
        
    # Train the LightGBM model - this does work
    # model = lgb.train(params, lgb_train, num_boost_round=var_n_boost_round, valid_sets=[lgb_val])
    model = lgb.train(params, lgb_train, valid_sets=[lgb_val])
    # ----------------------------------
    
    # Evaluate the LightGBM model on the (internal) validation set
    y_pred = model.predict(X_test)
    aucpr = average_precision_score(y_test, y_pred)
    return aucpr

In [ ]:
from optuna.logging import set_verbosity

# Set the logging level to WARNING
set_verbosity(optuna.logging.ERROR)

var_n_trials = var_n_boost_round

study = optuna.create_study(direction="maximize")
# https://optuna.readthedocs.io/en/stable/reference/logging.html
optuna.logging.set_verbosity(optuna.logging.ERROR)
study.optimize(objective, n_trials=var_n_trials)


print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
    
   

In [ ]:
best_trial = study.best_trial
best_params = best_trial.params

# Set up the logger to write to a file
logger = logging.getLogger()
handler = logging.FileHandler(var_path_model + var_model_name_full + "_best_parameters_apply.log")
logger.addHandler(handler)

# Train the LightGBM model using the best hyperparameters
# best_model = lgb.train(best_params, lgb_train, num_boost_round=var_n_boost_round, valid_sets=[lgb_val])
best_model = lgb.train(best_params, lgb_train, valid_sets=[lgb_val])

# Reverse the logging to the file on disc
logger.removeHandler(handler)
handler.close() 

In [ ]:
# show the best parameters
print(best_model.params)

In [ ]:
# evaluate the best model on the test data
y_pred = best_model.predict(X_test)

# evaluate the initial values based on the (internal Test data)
auc_pred = roc_auc_score(y_test, y_pred, average='weighted')
print(f'Test AUC: {auc_pred:.4f}')

aucpr = average_precision_score(y_test, y_pred, average='weighted', pos_label=1)
print(f'Test AUCPR: {aucpr:.4f}')


In [ ]:
import pickle
# set the path for the pickel file
path_model = var_path_model + var_model_name_full + "_model_stored.pkl"
# Save object as pickle file
pickle.dump(best_model, open(path_model, 'wb'), pickle.HIGHEST_PROTOCOL)

In [ ]:
# Get the feature importance
importance = best_model.feature_importance()

# Store the feature importance and column names in a Pandas DataFrame
feature_imp = pd.DataFrame({"Feature": X_test.columns, "Value": importance})
feature_imp = feature_imp.sort_values(by='Value', ascending=False, na_position='last')

feature_imp = feature_imp.reset_index(drop=True)
feature_imp['Feature_Rank'] = feature_imp.index

In [ ]:
feature_imp.head(25)

In [ ]:
feature_imp.to_parquet(var_path_model + var_model_name_full + "_feature_importance.parquet", compression='gzip')

In [ ]:
type(y_pred)

In [ ]:
# store the variales list as dictionary in a JSON file to read back later

v_variable_list = {
    "var_model_name": var_model_name,
    "var_model_name_full": var_model_name_full,
    "num_cols": num_cols,
    "cat_cols": cat_cols,
    "rest_cols": rest_cols,
    "label": label,
    "features": features,
    "excluded_features": excluded_features,
    "Test_AUC": f'{auc_pred:.5f}',
    "Test_AUCPR": f'{aucpr:.5f}'
}

In [ ]:
print(v_variable_list)

In [ ]:
type(v_variable_list)

In [ ]:
# Write the dictionary to a JSON file
with open(var_path_model + var_model_name_full + "_variable_list.json", "w") as f:
    json.dump(v_variable_list, f)

In [ ]:
# show the best parameters
best_parameters = best_model.params
print(best_parameters)

In [ ]:
with open(var_path_model + var_model_name_full + "_model_parameters.txt", "w") as file:
    print(best_parameters, file=file)

In [ ]:
df_best_parameters = pd.read_csv(var_path_model + var_model_name_full + "_model_parameters.txt", sep = "@",header = None, names=["lgbm_parameters"])

In [ ]:
df_best_parameters.head()

In [ ]:
# Path for Image
var_path_opt_history_png = var_path_model + var_model_name_full + "_opt_history.png"

# Plot the optimization history and save to a file
fig = plot_optimization_history(study)
fig.write_image(var_path_opt_history_png)    

# Plot the hyperparameter importances - this pictures might get large so maybe look at them in the PNG file
# plot_optimization_history(study)

In [ ]:
# Path for Image
var_path_param_importances_png = var_path_model + var_model_name_full + "_param_importances.png"

# Plot the hyperparameter importances and save to a file
fig_para = plot_param_importances(study)
fig_para.write_image(var_path_param_importances_png)

# Plot the hyperparameter importances - this pictures might get large so maybe look at them in the PNG file
# plot_param_importances(study)

## Apply the LightGBM model with all the settings

In [ ]:
import pickle
# set the path for the pickel file
path_apply_model = var_path_model + var_model_name_full + "_model_stored.pkl"

clf_apply = pickle.load(open(path_apply_model, 'rb'))

In [ ]:
import json

# Read the JSON file back into a Python dictionary
with open(var_path_model + var_model_name_full + "_variable_list.json", "r") as f:
    loaded_dict = json.load(f)

# fill the list of categorical columns
new_cat_cols = loaded_dict['cat_cols']
new_features = loaded_dict['features']

In [ ]:
print(new_cat_cols)

In [ ]:
df_test_apply = data_test[new_features].copy()
df_test_apply[new_cat_cols] = df_test_apply[new_cat_cols].astype('category')

In [ ]:
# Get the predicted probabilities for each class
probabilities = clf_apply.predict(df_test_apply)
# prediction = clf_apply.predict(df_test_apply)

In [ ]:
probabilities_df = pd.DataFrame(probabilities, columns = ['P1'])
# prediction_df = pd.DataFrame(prediction, columns = ['Target_pred'])

In [ ]:
probabilities_df.head()

In [ ]:
# Join the original target column with the predicted probabilities
result = pd.concat([data_test, probabilities_df], axis=1)

In [ ]:
result.head()

In [ ]:
result.to_parquet(var_path_data + var_model_name_full + "_scored_test_data.parquet", compression='gzip')

In [ ]:
# evaluate the best model on the test data
auc_pred = roc_auc_score(result['Target'], result['P1'], average='weighted')
print(f'Test AUC: {auc_pred:.4f}')

In [ ]:
# from sklearn.metrics import average_precision_score
aucpr = average_precision_score(result['Target'], result['P1'], average='weighted', pos_label='1')
print(f'Test AUCPR: {aucpr:.4f}')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

y_true = result["Target"].astype(int).values
y_score = result["P1"].values

precision, recall, thresholds = precision_recall_curve(y_true, y_score)
auc_pr = np.trapz(precision, recall)

plt.tight_layout()
plt.gcf().set_size_inches(10.24, 7.68)
plt.step(recall, precision, color='b', alpha=0.2, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall curve: AUCPR={0:0.4f}'.format(auc_pr))
plt.show()